# Train with the Test split?
This is what we want to do:

1. predict batches with ...
.TODO: create list of content

### 1.) set run-config and hyperparameters

In [1]:
FINAL_SUBMISSION = False # will perform a test on a validation split if set to False

TEST_BATCH_SIZE = 200 # Number of Test entries to add to the training set for the next iteration
ITER_PRINT_EVERY = 2 # Which Iterations to print (every nth)

In [2]:
%run ../notebooks/utils.ipynb

### 2.) import python modules

In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
import time
import warnings
warnings.filterwarnings("ignore")

### 3.) define helper functions

In [4]:
def calc_scores(y_test, y_pred):
    y_test_tmp = y_test.copy()
    accuracy = metrics.accuracy_score(y_test_tmp, y_pred)
    confusion_matrix = (metrics.confusion_matrix(y_test_tmp, y_pred)).tolist()
    dmc_score = np.sum(confusion_matrix * np.array([[0, -25], [-5, 5]]))
    return accuracy, dmc_score, confusion_matrix

def get_classifier(name):
    return {
            'xgb': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bytree=1, disable_default_eval_metric=1,eval_metric='aucpr',
                gamma=1.8285912697052542, reg_lambda=0.4149772770711012,
                max_bin=254, max_delta_step=7.2556696256684035,
                max_depth=3, min_child_weight=1.0317712458399741, missing=None,
                n_estimators=445, n_jobs=-1, objective='binary:logistic', random_state=0, reg_alpha=0,
                scale_pos_weight=1,silent=True,
                subsample=1, tree_method='gpu_hist', verbosity=2, seed=42),
            'svc': SVC(C=56.98164719395536, cache_size=8000, class_weight=None, coef0=0.0,  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
                kernel='linear', max_iter=-1, probability=True, random_state=None,
                shrinking=True, tol=0.002848504943774676, verbose=0)}[name]

def get_best_classifier_for_sample(idx, validation_set):
    ground_truth = validation_set.iloc[idx].fraud
    
    lsvc_predict = validation_set.iloc[idx].lsvc_predict
    xgb_predict = validation_set.iloc[idx].xgb_predict
    lsvc_proba = validation_set.iloc[idx].lsvc_proba
    xgb_prob = validation_set.iloc[idx].xgb_proba
    
    
    # Both classifier predicted the calue correctly
    if (lsvc_predict == ground_truth) and (xgb_predict == ground_truth):
        if (lsvc_proba + 0.025) > xgb_prob:
            return "lsvc"
        else:
            return "xgboost"
    # lsvc predicted correctly
    elif (validation_set.iloc[idx].lsvc_predict == ground_truth) and (xgb_predict != ground_truth):
        return "lsvc"
    
    # xgboost predicted correcltly
    elif (lsvc_predict != ground_truth) and (xgb_predict == ground_truth):
        return "xgboost"
    
    # If No classifier predicted the knn correct, None is returned
    else: 
        return ground_truth.value

### KNN Search Class

In [5]:
class KNNLookup():

    def __init__(self, knn_data):

        #self.knn = NearestNeighbors(n_neighbors=1)
        #self.knn.fit(knn_data.values.tolist())

        self.tree = KDTree(knn_data)

    def refit(self, knn_data):
        self.tree = KDTree(knn_data)

    def find_nearest_neighbor(self, row_scaled, dataset_scaled):
        diffs = [np.sum((row_scaled - ds_row) ** 2) for idx, ds_row in dataset_scaled.iterrows()]
        idx = np.argmin(diffs)
        return idx, diffs[idx]

    def find_nearest_neighbor2(self, row_scaled, dataset_scaled):
        dist, ind = self.tree.query([row_scaled.values], k=1)
        return np.ravel(ind)[0], np.ravel(dist)[0]

### 4.) Import Data

In [6]:
trainandknn_Xy_original_df = pd.read_csv("../data/train.csv", sep="|") if FINAL_SUBMISSION else pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:4000] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")

#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")

### 5.) Prepare Input X and Label Y Data

In [7]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

### 6.) DataTransformer Class and data transformation

In [8]:
class DataTransformer:
    """
    for scaling, data transformations (new features, one-hot encoding, categorical, ...)
    """
    
    def __init__(self, scaler):
        self.scaler = scaler

    def fit_scaler(self, df):
        df_tmp = df.copy()
        self.scaler.fit(df_tmp.astype(np.float64))
        return self
        
    def apply_scaler(self, df):
        df_temp = df.copy()
        return pd.DataFrame(self.scaler.transform(df_temp),df_temp.index, df_temp.columns)
    
    def inverse_scale(self, df):
        df_tmp = df.copy()
        return pd.DataFrame(self.scaler.inverse_transform(df_tmp), df_tmp.index, df_tmp.columns)
    
    def add_features(self,df):
        #TODO: Choose relevant features
        df_tmp = df.copy()
        df_tmp['totalScannedLineItems'] = df_tmp['scannedLineItemsPerSecond'] * df_tmp['totalScanTimeInSeconds']
        #df['avgTimePerScan'] = 1/ df['scannedLineItemsPerSecond']
        #df['avgValuePerScan'] = df['avgTimePerScan'] * df['valuePerSecond']
        #df['withoutRegisPerPosition'] = df['scansWithoutRegistration'] / df['totalScannedLineItems'] #equivalent to lineItemVoidsPerPosition?
        #df['quantiModPerPosition'] = df['quantityModifications'] / df['totalScannedLineItems']
        #df['lineItemVoidsPerTotal'] = df['lineItemVoids'] / df['grandTotal']
        #df['withoutRegisPerTotal'] = df['scansWithoutRegistration'] / df['grandTotal']
        #df['quantiModPerTotal'] = df['quantityModifications'] / df['grandTotal']
        #df['lineItemVoidsPerTime'] = df['lineItemVoids'] / df['totalScanTimeInSeconds']
        #df['withoutRegisPerTime'] = df['scansWithoutRegistration'] / df['totalScanTimeInSeconds']
        #df['quantiModPerTime'] = df['quantityModifications'] / df['totalScanTimeInSeconds']
        #df['valuePerScannedLineItem'] = df['valuePerSecond'] / df['scannedLineItemsPerSecond']
        return df_tmp
    
    def transform(self, df):
        """
        All in one: Apply all transform methods
            1.) addFeatures
            2.) apply_scaler
        """
        df_tmp = df.copy()
        return self.apply_scaler(self.add_features(df_tmp))
    


In [9]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()
train_complete_y_df = train_complete_y_originial_df.copy()

test_final_X_df = transformer.add_features(test_final_X_df)


test_X_scaled_df.head(2)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
0,0.317174,-0.857599,1.329772,-0.418354,0.959638,0.869131,-0.085073,-0.021819,-0.110333,-0.98995
1,-0.276669,0.157498,0.306417,0.451036,0.324238,-0.884543,-0.068487,-0.094210,-0.352573,1.33811


| Name     | New Features | Scaled |
|----------|--------------|--------|
| orig     |      [ ]     |   [ ]  |
| unscaled |      [X]     |   [ ]  |
| scaled   |      [X]     |   [X]  |

### 5 1/2.) train normally with all available classifiers for classifying knn split

In [10]:
knnwithprobs_Xy_df = knn_X_unscaled_df.copy()
knnwithprobs_Xy_df['fraud'] = knn_y_df
knnwithprobs_Xy_df = knnwithprobs_Xy_df.reset_index(drop=True)
knnwithprobs_Xy_df.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems,fraud
0,4,923,93.36,2,2,3,0.018418,0.101148,0.117647,17.0,0
1,6,1473,0.01,10,8,0,0.005431,0.000007,1.250000,8.0,0
2,6,575,8.55,5,1,0,0.050435,0.014870,0.172414,29.0,0
3,4,1519,90.38,8,5,2,0.007900,0.059500,0.666667,12.0,0
4,4,29,84.08,1,0,4,0.724138,2.899310,0.047619,21.0,0


In [11]:
def test_routine(data_dict, data_transformer, knn_lookup):
    pltrain_X_unscaled_df = data_dict['pltrain_X_unscaled_df']
    pltrain_X_scaled_df = transformer.apply_scaler(pltrain_X_unscaled_df)
    pltrain_y_df = data_dict['pltrain_y_df']

    lsvc_pltrain = get_classifier("svc")
    xgb_pltrain = get_classifier("xgb")

    lsvc_pltrain.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)
    xgb_pltrain.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)

    # Predict on ValidationSet
    lsvc_pred_val = lsvc_pltrain.predict(data_dict['val_X_scaled_df'].values)
    xgb_pred_val = xgb_pltrain.predict(data_dict['val_X_unscaled_df'].values)
    lsvc_prob_val = lsvc_pltrain.predict_proba(data_dict['val_X_scaled_df'].values)
    xgb_prob_val = xgb_pltrain.predict_proba(data_dict['val_X_unscaled_df'].values)

    own_classifier_pred_val = classify(xgb_pltrain, lsvc_pltrain,  data_dict['val_X_unscaled_df'],
                                       data_dict['knnwithprob_Xy_unscaled_df'], data_transformer, knn_lookup)
    
    val_pred_Xy = data_dict['val_X_unscaled_df'].copy()
    val_pred_Xy['fraud'] = data_dict['val_y_df'].copy()
    val_pred_Xy['xgb_predict'] = xgb_pred_val
    val_pred_Xy['lsvc_predict'] = lsvc_pred_val
    val_pred_Xy['xgb_proba'] = [round(max(x), 3) for x in xgb_prob_val]
    val_pred_Xy['lsvc_proba'] = [round(max(x), 3) for x in lsvc_prob_val]
    val_pred_Xy['own_predict'] = own_classifier_pred_val

    

    lsvc_val_acc, lsvc_val_dmc, lsvc_val_conf_mat = calc_scores(data_dict['val_y_df'].values, lsvc_pred_val)
    xgb_val_acc, xgb_val_dmc, xgb_val_conf_mat = calc_scores(data_dict['val_y_df'].values, xgb_pred_val)
    own_classifier_val_acc, own_classifier_val_dmc, own_classifier_val_conf_mat = calc_scores(
        data_dict['val_y_df'], own_classifier_pred_val)

    lsvc_pltrain = get_classifier("svc")
    xgb_pltrain = get_classifier("xgb")

    lsvc_pltrain.fit(data_dict['test_X_scaled_df'].values, data_dict['test_y_df'].values)
    xgb_pltrain.fit(data_dict['test_X_unscaled_df'].values, data_dict['test_y_df'].values)

    # Predict on original full size (~1900 samples) just trained on test_data
    lsvc_pred_train = lsvc_pltrain.predict(data_dict['train_complete_X_scaled_df'].values)
    xgb_pred_train = xgb_pltrain.predict(data_dict['train_complete_X_unscaled_df'].values)
    lsvc_prob_train = lsvc_pltrain.predict_proba(data_dict['train_complete_X_scaled_df'].values)
    xgb_prob_train = xgb_pltrain.predict_proba(data_dict['train_complete_X_unscaled_df'].values)
    
    own_classifier_pred_train = classify(xgb_pltrain,lsvc_pltrain, data_dict['train_complete_X_unscaled_df'],
                                         data_dict['knnwithprob_Xy_unscaled_df'], data_transformer, knn_lookup)
    
    
    
    train_pred_complete_Xy = data_dict['train_complete_X_unscaled_df'].copy()
    train_pred_complete_Xy['fraud'] = data_dict['train_complete_y_df'].copy()
    train_pred_complete_Xy['xgb_predict'] = xgb_pred_train
    train_pred_complete_Xy['lsvc_predict'] = lsvc_pred_train
    train_pred_complete_Xy['xgb_prob'] = [round(max(x), 3) for x in xgb_prob_train]
    train_pred_complete_Xy['lsvc_prob'] = [round(max(x), 3) for x in lsvc_prob_train]
    train_pred_complete_Xy['own_predict'] = own_classifier_pred_train
    
    
    
    lsvc_train_acc, lsvc_train_dmc, lsvc_train_conf_mat = calc_scores(data_dict['train_complete_y_df'], lsvc_pred_train)
    xgb_train_acc, xgb_train_dmc, xgb_train_conf_mat = calc_scores(data_dict['train_complete_y_df'], xgb_pred_train)
    own_classifier_train_acc, own_classifier_train_dmc, own_classifier_train_conf_mat = calc_scores(
        data_dict['train_complete_y_df'], own_classifier_pred_train)

    results = {"lin_svc": {
        "val": {
            "dmc_score": lsvc_val_dmc,
            "conf_matrix": lsvc_val_conf_mat
        },
        "train": {
            "dmc_score": lsvc_train_dmc,
            "conf_matrix": lsvc_train_conf_mat
        }
    },
        "xgboost": {
            "val": {
                "dmc_score": xgb_val_dmc,
                "conf_matrix": xgb_val_conf_mat
            },
            "train": {
                "dmc_score": xgb_train_dmc,
                "conf_matrix": xgb_train_conf_mat
            }
        },
        "own_classifier": {
            "val": {
                "dmc_score": own_classifier_val_dmc,
                "conf_matrix": own_classifier_val_conf_mat
            },
            "train": {
                "dmc_score": own_classifier_train_dmc,
                "conf_matrix": own_classifier_train_conf_mat
            }
        },

    }
    return results, train_pred_complete_Xy.copy(), val_pred_Xy.copy()

In [24]:
def test_routine_without_knn(data_dict, data_transformer):
    pltrain_X_unscaled_df = data_dict['pltrain_wo_knn_X_unscaled_df']
    pltrain_X_scaled_df = transformer.apply_scaler(pltrain_X_unscaled_df)
    pltrain_y_df = data_dict['pltrain_wo_knn_y_df']
    
    
    pred_val_time = time.time()
    lsvc_pltrain = get_classifier("svc")
    xgb_pltrain = get_classifier("xgb")

    lsvc_pltrain.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)
    xgb_pltrain.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)

    # Predict on ValidationSet
    lsvc_pred_val = lsvc_pltrain.predict(data_dict['val_X_scaled_df'].values)
    xgb_pred_val = xgb_pltrain.predict(data_dict['val_X_unscaled_df'].values)
    lsvc_prob_val = lsvc_pltrain.predict_proba(data_dict['val_X_scaled_df'].values)
    xgb_prob_val = xgb_pltrain.predict_proba(data_dict['val_X_unscaled_df'].values)

    own_classifier_pred_val = classify_without_knn(xgb_pltrain, lsvc_pltrain, data_dict['val_X_unscaled_df'], data_transformer)
    
    pred_val_end = time.time()
    print("Prediction on val ", pred_val_end- pred_val_time)
    
    start = time.time()
    val_pred_Xy = data_dict['val_X_unscaled_df'].copy()
    val_pred_Xy['fraud'] = data_dict['val_y_df'].copy()
    val_pred_Xy['xgb_predict'] = xgb_pred_val
    val_pred_Xy['lsvc_predict'] = lsvc_pred_val
    val_pred_Xy['xgb_proba'] = [round(max(x), 3) for x in xgb_prob_val]
    val_pred_Xy['lsvc_proba'] = [round(max(x), 3) for x in lsvc_prob_val]
    val_pred_Xy['own_predict'] = own_classifier_pred_val

    

    lsvc_val_acc, lsvc_val_dmc, lsvc_val_conf_mat = calc_scores(data_dict['val_y_df'].values, lsvc_pred_val)
    xgb_val_acc, xgb_val_dmc, xgb_val_conf_mat = calc_scores(data_dict['val_y_df'].values, xgb_pred_val)
    own_classifier_val_acc, own_classifier_val_dmc, own_classifier_val_conf_mat = calc_scores(
        data_dict['val_y_df'], own_classifier_pred_val)
    print("Calculating score val: ", time.time()- start)
    
    start_train = time.time()
    
    lsvc_pltrain = get_classifier("svc")
    xgb_pltrain = get_classifier("xgb")

    lsvc_pltrain.fit(data_dict['test_wo_knn_X_scaled_df'].values, data_dict['test_wo_knn_y_df'].values)
    xgb_pltrain.fit(data_dict['test_wo_knn_X_unscaled_df'].values, data_dict['test_wo_knn_y_df'].values)

    # Predict on original full size (~1900 samples) just trained on test_data
    lsvc_pred_train = lsvc_pltrain.predict(data_dict['train_complete_X_scaled_df'].values)
    xgb_pred_train = xgb_pltrain.predict(data_dict['train_complete_X_unscaled_df'].values)
    lsvc_prob_train = lsvc_pltrain.predict_proba(data_dict['train_complete_X_scaled_df'].values)
    xgb_prob_train = xgb_pltrain.predict_proba(data_dict['train_complete_X_unscaled_df'].values)
       
    own_classifier_pred_train = classify_without_knn(xgb_pltrain, lsvc_pltrain, data_dict['train_complete_X_unscaled_df'], data_transformer)
    
    print("Prediction on train ", time.time()- start_train)

    start_calc_time = time.time()
    train_pred_complete_Xy = data_dict['train_complete_X_unscaled_df'].copy()
    train_pred_complete_Xy['fraud'] = data_dict['train_complete_y_df'].copy()
    train_pred_complete_Xy['xgb_predict'] = xgb_pred_train
    train_pred_complete_Xy['lsvc_predict'] = lsvc_pred_train
    train_pred_complete_Xy['xgb_prob'] = [round(max(x), 3) for x in xgb_prob_train]
    train_pred_complete_Xy['lsvc_prob'] = [round(max(x), 3) for x in lsvc_prob_train]
    train_pred_complete_Xy['own_predict'] = own_classifier_pred_train
    
    
    
    lsvc_train_acc, lsvc_train_dmc, lsvc_train_conf_mat = calc_scores(data_dict['train_complete_y_df'], lsvc_pred_train)
    xgb_train_acc, xgb_train_dmc, xgb_train_conf_mat = calc_scores(data_dict['train_complete_y_df'], xgb_pred_train)
    own_classifier_train_acc, own_classifier_train_dmc, own_classifier_train_conf_mat = calc_scores(
        data_dict['train_complete_y_df'], own_classifier_pred_train)
    
    print("Calculating score train: ", time.time()- start_calc_time)

    results = {"lin_svc": {
        "val": {
            "dmc_score": lsvc_val_dmc,
            "conf_matrix": lsvc_val_conf_mat
        },
        "train": {
            "dmc_score": lsvc_train_dmc,
            "conf_matrix": lsvc_train_conf_mat
        }
    },
        "xgboost": {
            "val": {
                "dmc_score": xgb_val_dmc,
                "conf_matrix": xgb_val_conf_mat
            },
            "train": {
                "dmc_score": xgb_train_dmc,
                "conf_matrix": xgb_train_conf_mat
            }
        },
        "own_classifier": {
            "val": {
                "dmc_score": own_classifier_val_dmc,
                "conf_matrix": own_classifier_val_conf_mat
            },
            "train": {
                "dmc_score": own_classifier_train_dmc,
                "conf_matrix": own_classifier_train_conf_mat
            }
        },

    }
    return results, train_pred_complete_Xy.copy(), val_pred_Xy.copy()

In [13]:
def classify(xgboost_fitted, linear_svc_fitted, data_to_predict, data_knn_with_probs, transformer, knn_lookup):
    prediction = []
    data_knn_X_scaled = transformer.apply_scaler(
        data_knn_with_probs.copy().drop(columns=["fraud", "xgb_predict", "xgb_proba", "lsvc_predict", "lsvc_proba"]))
    data_to_predict_scaled = transformer.apply_scaler(data_to_predict.copy())
    data_to_predict_unscaled = data_to_predict.copy()

    knn_time = []
    pred_time = []
    tolerance = 0.5
    
    #Check which scaler was used for preprocessing
    if str(type(transformer.scaler)) == "<class 'sklearn.preprocessing.data.StandardScaler'>":
        tolerance = 1.4
    
    #print(len(data_to_predict_unscaled))
    for i, row in data_to_predict_unscaled.iterrows():
        #print(i)
        if row.trustLevel >= 3:
            prediction.append(0)

        else:

            start_knn = time.time()
            idx_knn, distance_knn = knn_lookup.find_nearest_neighbor2(data_to_predict_scaled.iloc[i], data_knn_X_scaled)
            knn_time.append(time.time() - start_knn)
            # If distance to knn is to big, classify them directly
            if distance_knn > tolerance:

                pred_start = time.time()
                xgb_pred = xgboost_fitted.predict([data_to_predict_unscaled.iloc[i].values])[0]
                xgb_prob = max(np.ravel(xgboost_fitted.predict_proba([data_to_predict_unscaled.iloc[i].values])))

                lsvc_pred = linear_svc_fitted.predict([data_to_predict_scaled.iloc[i]])[0]
                lsvc_prob = max(np.ravel(linear_svc_fitted.predict_proba([data_to_predict_scaled.iloc[i].values])))

                pred_time.append(time.time() - pred_start)
                # If both classified them equal, take one of both
                if xgb_pred == lsvc_pred:
                    prediction.append(xgb_pred)

                # if classification is not equal, take the one with higher probability
                elif xgb_prob > lsvc_prob:
                    prediction.append(xgb_pred)
                else:
                    prediction.append(lsvc_pred)

            # If distance is smaller than 0.15, use knn    
            else:
                best_classifier = get_best_classifier_for_sample(idx_knn, data_knn_with_probs)
                if isinstance(best_classifier, int):
                    prediction.append(best_classifier)
                    
                elif best_classifier == "xgboost":
                    pred_start = time.time()
                    prediction.append(xgboost_fitted.predict([data_to_predict_unscaled.iloc[i].values])[0])
                    pred_end = time.time()


                elif best_classifier == "lsvc":
                    pred_start = time.time()
                    prediction.append(linear_svc_fitted.predict([data_to_predict_scaled.iloc[i].values])[0])
                    pred_end = time.time()

                pred_time.append(pred_end - pred_start)
    return pd.DataFrame({"fraud": prediction})

In [19]:
def classify_without_knn(xgboost_fitted, linear_svc_fitted, data_to_predict, transformer):
    prediction = []
    data_to_predict_scaled = transformer.apply_scaler(data_to_predict.copy())
    data_to_predict_unscaled = data_to_predict.copy()
    pred_time, comp_time =[], []
    threshhold = 25/35
    
    #print(len(data_to_predict_unscaled))
    for i, row in data_to_predict_unscaled.iterrows():
        #print(i)
        
        if row.trustLevel >= 3:
            prediction.append(0)

        else:
            start_pred = time.time()
            xgb_pred = xgboost_fitted.predict([data_to_predict_unscaled.iloc[i].values])[0]
            xgb_prob = max(np.ravel(xgboost_fitted.predict_proba([data_to_predict_unscaled.iloc[i].values])))

            lsvc_pred = linear_svc_fitted.predict([data_to_predict_scaled.iloc[i]])[0]
            lsvc_prob = max(np.ravel(linear_svc_fitted.predict_proba([data_to_predict_scaled.iloc[i].values])))
            
            end_pred = time.time()
            
            prob_mean = np.mean(xgb_prob + lsvc_prob)
            
            if (lsvc_pred == 1) and (lsvc_prob > threshhold):
                prediction.append(lsvc_pred)
                continue
            
            elif (xgb_pred == 1) and (prob_mean > threshhold):
                prediction.append(xgb_pred)
                continue
            
            elif xgb_prob < (lsvc_prob + 0.025):
                prediction.append(lsvc_pred)
                continue
            
            else:
                prediction.append(xgb_pred)
            
            end_comparision = time.time()
            
            pred_time.append(end_pred- start_pred)
            comp_time.append(end_comparision-end_pred)
            
    print("Prediction Time. {} --- Comparison Time: {}".format(np.mean(pred_time), np.mean(comp_time)))
    return pd.DataFrame({"fraud": prediction})

In [ ]:
def semi_supervised_learning_procedure(test_X_unscaled,train_X_unscaled, train_y, test_data_dict, transformer):
    test_data_dict = test_data_dict
    pred_dfs = []
    pred_val_dfs = []
    
    pred_train_wo_knn_dfs = []
    pred_val_wo_knn_dfs = []
    results_wo_knn = []
    
    # initialize pseudo labeled train (pltrain) dataset with the transformed training data
    pltrain_X_unscaled_df = train_X_unscaled.copy()
    train_X_scaled_df = transformer.apply_scaler(train_X_unscaled)

    pltrain_wo_knn_X_unscaled_df = test_data_dict['train_X_wo_knn_unscaled_df'].copy()
    pltrain_wo_knn_X_scaled_df = transformer.apply_scaler(pltrain_wo_knn_X_unscaled_df)
    pltrain_wo_knn_y_df = test_data_dict['train_y_wo_knn_df']
    
    
    pltrain_y_df = train_y.copy()
    train_X_scaled_len = len(train_X_unscaled)
    train_wo_knn_X_scaled_len = len(pltrain_wo_knn_X_unscaled_df)
    print("{} available train data before pseudo labeling".format(train_X_scaled_len))

    train_X_scaled_df = transformer.apply_scaler(train_X_unscaled)

    linear_svc_initial = get_classifier('svc')
    linear_svc_initial.fit(train_X_scaled_df.values, pltrain_y_df.values)
    xgboost_initial = get_classifier('xgb')
    xgboost_initial.fit(train_X_unscaled.values, pltrain_y_df.values)
    
    knnwithprob_Xy_unscaled_df = test_data_dict['knnwithprob_Xy_unscaled_df']
    knnwithprobs_X_unscaled_df = knnwithprob_Xy_unscaled_df.copy().drop(columns=['fraud'])
    knnwithprobs_X_scaled_df = transformer.apply_scaler(knnwithprobs_X_unscaled_df.copy())
    knn_lookup = KNNLookup(knnwithprobs_X_unscaled_df)
    
    knnwithprob_Xy_unscaled_df['xgb_predict'] = xgboost_initial.predict(knnwithprobs_X_unscaled_df.values)
    knnwithprob_Xy_unscaled_df['xgb_proba'] = [round(max(x), 3) for x in
                                       xgboost_initial.predict_proba(knnwithprobs_X_unscaled_df.values)]
    knnwithprob_Xy_unscaled_df['lsvc_predict'] = linear_svc_initial.predict(knnwithprobs_X_scaled_df.values)
    knnwithprob_Xy_unscaled_df['lsvc_proba'] = [round(max(x), 3) for x in
                                        linear_svc_initial.predict_proba(knnwithprobs_X_scaled_df.values)]
    knn_dataframes = [knnwithprob_Xy_unscaled_df.copy()]
    results = []
    for i in range(TEST_BATCH_SIZE, len(test_X_unscaled), TEST_BATCH_SIZE):
        if int(i / TEST_BATCH_SIZE) % ITER_PRINT_EVERY == 0:
            print("iteration", int(i / TEST_BATCH_SIZE), "\t/", int(np.ceil(len(test_X_unscaled) / TEST_BATCH_SIZE)),
                  "with batch from", i - TEST_BATCH_SIZE, "\t to", i, ", training with", len(pltrain_y_df), "samples")
        # get batch from test set
        testbatch_X_df = test_X_unscaled.iloc[i - TEST_BATCH_SIZE:i].copy().reset_index(drop=True)
                
        """
        ----------------------------------------------------OLD PART ------------------------------------------------------------
        """     
        """
        # extend pseudo labeled train (pltrain) dataset by predicting the batch
        pltrain_X_unscaled_df, pltrain_y_df = get_extended_pltrain_for_batch(testbatch_X_df, pltrain_X_unscaled_df,
                                                                             pltrain_y_df, knnwithprob_Xy_unscaled_df,
                                                                             transformer, knn_lookup)
        pltrain_X_scaled_df = transformer.apply_scaler(pltrain_X_unscaled_df)
        test_data_dict['pltrain_X_unscaled_df'] = pltrain_X_unscaled_df.copy()
        test_data_dict['pltrain_X_scaled_df'] = pltrain_X_scaled_df.copy()
        test_data_dict['pltrain_y_df'] = pltrain_y_df.copy()
        
        
        linear_svc = get_classifier('svc')
        linear_svc.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)

        xgboost = get_classifier('xgb')
        xgboost.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)

        knnwithprob_Xy_unscaled_df_tmp = knnwithprob_Xy_unscaled_df.copy().drop(
            columns=['fraud', 'xgb_predict', 'xgb_proba', 'lsvc_predict', 'lsvc_proba'])
        
        #own preds on knn for DEBUG
        own_preds = classify(xgboost, linear_svc,knnwithprob_Xy_unscaled_df_tmp.copy(), knnwithprob_Xy_unscaled_df, transformer, knn_lookup)
        
        knnwithprob_Xy_unscaled_df['xgb_predict'] = xgboost.predict(knnwithprob_Xy_unscaled_df_tmp.values)
        knnwithprob_Xy_unscaled_df['xgb_proba'] = [round(max(x), 3) for x in xgboost.predict_proba(knnwithprob_Xy_unscaled_df_tmp.values)]
        knnwithprob_Xy_unscaled_df['lsvc_predict'] = linear_svc.predict(transformer.apply_scaler(knnwithprob_Xy_unscaled_df_tmp).values)
        knnwithprob_Xy_unscaled_df['lsvc_proba'] = [round(max(x), 3) for x in
                                            linear_svc.predict_proba(transformer.apply_scaler(knnwithprob_Xy_unscaled_df_tmp).values)]
        
        knnwithprob_Xy_unscaled_df_copy = knnwithprob_Xy_unscaled_df.copy()
        knnwithprob_Xy_unscaled_df_copy['own_predict'] = own_preds
        knn_dataframes.append(knnwithprob_Xy_unscaled_df_copy)
        test_data_dict['knnwithprob_Xy_unscaled_df'] = knnwithprob_Xy_unscaled_df
        
        # Only Test from PLTrain without the data from train
        test_data_dict['test_X_unscaled_df'] = pltrain_X_unscaled_df.iloc[train_X_scaled_len:]
        test_data_dict['test_X_scaled_df'] = pltrain_X_scaled_df.iloc[train_X_scaled_len:]
        test_data_dict['test_y_df'] = pltrain_y_df.iloc[train_X_scaled_len:]
        
        res, pred_df, pred_val_df = test_routine(test_data_dict, transformer, knn_lookup)
        pred_dfs.append(pred_df)
        pred_val_dfs.append(pred_val_df)
        print("XGBoost: PLTrain auf Val: {} --- PLTest auf Train: {} || LinearSVC:  PLTrain auf Val: {} --- PLTest auf Train: {} || Own Classifier:  PLTrain auf Val: {} --- PLTest auf Train: {}".format(
            res['xgboost']['val']['dmc_score'],res['xgboost']['train']['dmc_score'],res['lin_svc']['val']['dmc_score'],res['lin_svc']['train']['dmc_score'],res['own_classifier']['val']['dmc_score'],res['own_classifier']['train']['dmc_score']))
        
        results.append(res)
        """

        """
        ----------------------------------------------------NEW PART ------------------------------------------------------------
        """
        print("Next Batch starts")
        pltrain_wo_knn_X_unscaled_df, pltrain_wo_knn_y_df = get_extended_pltrain_for_batch_without_knn(testbatch_X_df, pltrain_wo_knn_X_unscaled_df,
                                                                             pltrain_wo_knn_y_df, transformer)
        pltrain_wo_knn_X_scaled_df = transformer.apply_scaler(pltrain_wo_knn_X_unscaled_df)
        test_data_dict['pltrain_wo_knn_X_unscaled_df'] = pltrain_wo_knn_X_unscaled_df.copy()
        test_data_dict['pltrain_wo_knn_X_scaled_df'] = pltrain_wo_knn_X_scaled_df.copy()
        test_data_dict['pltrain_wo_knn_y_df'] = pltrain_wo_knn_y_df.copy()
        
        
        linear_svc_wo_knn = get_classifier('svc')
        linear_svc_wo_knn.fit(pltrain_wo_knn_X_unscaled_df.values, pltrain_wo_knn_y_df.values)

        xgboost_wo_knn = get_classifier('xgb')
        xgboost_wo_knn.fit(pltrain_wo_knn_X_unscaled_df.values, pltrain_wo_knn_y_df.values)
        
        
        
        # Only Test from PLTrain without the data from train
        test_data_dict['test_wo_knn_X_unscaled_df'] = pltrain_wo_knn_X_unscaled_df.iloc[train_wo_knn_X_scaled_len:]
        test_data_dict['test_wo_knn_X_scaled_df'] = pltrain_wo_knn_X_scaled_df.iloc[train_wo_knn_X_scaled_len:]
        test_data_dict['test_wo_knn_y_df'] = pltrain_wo_knn_y_df.iloc[train_wo_knn_X_scaled_len:]
        
        res_wo_knn, pred_train_df_wo_knn, pred_val_df_wo_knn = test_routine_without_knn(test_data_dict, transformer)
        
        results_wo_knn.append(res_wo_knn)
        pred_train_wo_knn_dfs.append(pred_train_df_wo_knn)
        pred_val_wo_knn_dfs.append(pred_val_df_wo_knn)
        
        print("XGBoost: PLTrain auf Val: {} --- PLTest auf Train: {} || LinearSVC:  PLTrain auf Val: {} --- PLTest auf Train: {} || Own Classifier:  PLTrain auf Val: {} --- PLTest auf Train: {}".format(
            res_wo_knn['xgboost']['val']['dmc_score'],res_wo_knn['xgboost']['train']['dmc_score'],res_wo_knn['lin_svc']['val']['dmc_score'],res_wo_knn['lin_svc']['train']['dmc_score'],res_wo_knn['own_classifier']['val']['dmc_score'],res_wo_knn['own_classifier']['train']['dmc_score']))
        

    """
    # use last few rows that cant fill up a complete batch as a smaller batch
    print("iteration", int(i / TEST_BATCH_SIZE) + 1, "\twith batch from", i, "\t to", len(test_X_unscaled),
          ", training with", len(pltrain_X_unscaled_df), "samples")
    testbatch_X_transformed_df = test_X_unscaled.iloc[i:len(test_X_unscaled)]
    """
    xgb_final = get_classifier("xgb")
    lsvc_final = get_classifier("svc")
    
    xgb_final.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)
    lsvc_final.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)
    final_prediction = classify(xgb_final, lsvc_final, test_final_X_df.iloc[:10000], test_data_dict['knnwithprob_Xy_unscaled_df'], transformer, knn_lookup)
    
    xgb_final_wo_knn = get_classifier("xgb")
    lsvc_final_wo_knn = get_classifier("svc")
    
    xgb_final_wo_knn.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)
    lsvc_final_wo_knn.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)
    final_prediction_wo_knn = classify(xgb_final, lsvc_final, test_final_X_df.iloc[:10000], test_data_dict['knnwithprob_Xy_unscaled_df'], transformer, knn_lookup)
    
    
    return results, final_prediction, knn_dataframes, pred_dfs, pred_val_dfs, results_wo_knn, final_prediction_wo_knn,  pred_train_wo_knn_dfs, pred_val_wo_knn_dfs

### 6.) iterative model training using pseudo-labeling
predict batches of the test set, add them to the previous training set and use this new training set to predict the next batch.

In [16]:
def get_extended_pltrain_for_batch(testbatch_X_unscaled_df, pltrain_X_unscaled_df, pltrain_y_df,
                                   knnwithprobs_Xy_unscaled, transformer, knn_lookup):
    # train a classificator on the pseudo labeled train (pltrain) dataset
    pltrain_X_scaled_df = transformer.apply_scaler(pltrain_X_unscaled_df)
    linear_svc = get_classifier('svc')
    linear_svc.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)

    xgboost = get_classifier('xgb')
    xgboost.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)

    testbatch_y = classify(xgboost, linear_svc, testbatch_X_unscaled_df, knnwithprobs_Xy_unscaled, transformer,
                           knn_lookup)

    # add batch to pseudo labeled train (pltrain) dataset. needs to ignore index as ids in test also start with 0
    pltrainnew_X_unscaled_df = pltrain_X_unscaled_df.append(testbatch_X_unscaled_df, ignore_index=True)
    pltrainnew_y_df = pltrain_y_df.append(testbatch_y,
                                          ignore_index=True)  # pltrainnew_Xy_unscaled_df[['fraud']], ignore_index=True)

    return pltrainnew_X_unscaled_df, pltrainnew_y_df
    
    


In [28]:
def get_extended_pltrain_for_batch_without_knn(testbatch_X_unscaled_df, pltrain_X_unscaled_df, pltrain_y_df,transformer):
    start = time.time()
    # train a classificator on the pseudo labeled train (pltrain) dataset
    pltrain_X_scaled_df = transformer.apply_scaler(pltrain_X_unscaled_df)
    linear_svc = get_classifier('svc')
    linear_svc.fit(pltrain_X_scaled_df.values, pltrain_y_df.values)

    xgboost = get_classifier('xgb')
    xgboost.fit(pltrain_X_unscaled_df.values, pltrain_y_df.values)

    testbatch_y = classify_without_knn(xgboost, linear_svc, testbatch_X_unscaled_df, transformer)
    
    # add batch to pseudo labeled train (pltrain) dataset. needs to ignore index as ids in test also start with 0
    pltrainnew_X_unscaled_df = pltrain_X_unscaled_df.append(testbatch_X_unscaled_df, ignore_index=True)
    pltrainnew_y_df = pltrain_y_df.append(testbatch_y,
                                          ignore_index=True)  # pltrainnew_Xy_unscaled_df[['fraud']], ignore_index=True)
    print("Calculating batch takes ", time.time()-start)
    return pltrainnew_X_unscaled_df, pltrainnew_y_df
    
    


In [ ]:
#def semi_supervised_learning_procedure(test_X_unscaled, train_y, test_data_dict, transformer):


data_dict = dict()
data_dict['train_complete_X_unscaled_df'] = train_complete_X_unscaled_df.copy()
data_dict['train_complete_X_scaled_df'] = transformer.apply_scaler(train_complete_X_unscaled_df)
data_dict['train_complete_y_df'] = train_complete_y_df.copy()

data_dict['knnwithprob_Xy_unscaled_df'] = transformer.add_features(knnwithprobs_Xy_df.copy())
data_dict['val_X_unscaled_df'] = val_X_unscaled_df.copy()
data_dict['val_X_scaled_df'] = transformer.apply_scaler(val_X_unscaled_df)
data_dict['val_y_df'] = val_y_df.copy()
data_dict['train_X_wo_knn_unscaled_df'] = train_X_wo_knn_unscaled_df
data_dict['train_X_wo_knn_scaled_df'] = train_X_wo_knn_scaled_df
data_dict['train_y_wo_knn_df'] = train_y_wo_knn_df

# results, final_prediction, knn_dataframes, pred_dfs, pred_val_dfs, results_wo_knn, final_prediction_wo_knn,  pred_train_wo_knn_dfs, pred_val_wo_knn_dfs
res, fin_pred, knnp_reds, train_preds, val_preds, results_wo_knn, final_prediction_wo_knn,  pred_train_wo_knn_dfs, pred_val_wo_knn_dfs= semi_supervised_learning_procedure(test_X_unscaled_df, train_X_unscaled_df, train_y_df, data_dict, transformer)

In [ ]:
train_complete_X_unscaled_df.copy().describe()

In [ ]:
fin_pred.describe()

In [ ]:
plot_results_ssl(res)

In [ ]:
plot_results_ssl(results_wo_knn)

In [ ]:
val = pred_val_wo_knn_dfs[0]
val[val.fraud != val.own_predict]

In [ ]:
val[val.fraud != val.lsvc_predict]

In [ ]:
val[val.fraud != val.xgb_predict]

In [ ]:
knn2.describe()

In [ ]:
type(val_preds)

In [ ]:
val3 = val_preds[2]
display(val3[(val3.xgb_predict != val3.fraud)])
print("lsvc")
display(val3[(val3.lsvc_predict != val3.fraud)])


In [ ]:
val3 = val_preds[2].copy()
lst = []
for idx, row in val3.iterrows():
    if row.own_predict == 1:
        if ((row.xgb_proba + row.lsvc_proba)/2) >= (25/35) :
            lst.append(1)          
        else : lst.append(0)

    else : lst.append(0)

val3['custom_predict'] = lst
val3[(val3.fraud != val3.own_predict)]

In [ ]:
xgb_scaler = MinMaxScaler(feature_range=(0.5,1))
xgb_scaler.fit([val3.xgb_proba])
val3['xgb_scaled'] = np.ravel(xgb_scaler.transform([val3.xgb_proba]))

In [ ]:
val3.describe()

In [ ]:
val3[(val3.lsvc_predict == 1) & (val3.lsvc_proba >= (25/35))]

In [ ]:
val3[(val3.xgb_predict == 1) & (val3.xgb_proba >= (25/35))]

In [ ]:
train3 = train_preds[2]
knn3 = knnp_reds[2]
knn3[(knn3.xgb_predict != knn3.fraud) | (knn3.fraud != knn3.lsvc_predict)]

In [ ]:
train3 = train_preds[2]
knn3 = knnp_reds[2]

lst = []
for idx, row in knn3.iterrows():
    if row.own_predict == 1:
        if ((row.xgb_proba + row.lsvc_proba)/2) >= (25/35) :
            lst.append(1)          
        else : lst.append(0)

    else : lst.append(0)

knn3['custom_predict'] = lst

knn3[(knn3.xgb_predict != knn3.fraud) | (knn3.fraud != knn3.lsvc_predict)]

In [ ]:
knn3[(knn3.xgb_predict != knn3.fraud) | (knn3.fraud != knn3.lsvc_predict)]

In [ ]:
knn3[(knn3.xgb_predict != knn3.fraud) | (knn3.fraud != knn3.lsvc_predict)]

In [ ]:
train4 = train_preds[3]
knn4 = knnp_reds[3]
knn4[(knn4.xgb_predict != knn4.fraud) | (knn4.fraud != knn4.lsvc_predict)]

In [ ]:
train1[(train1.xgb_predict != train1.fraud) | (train1.fraud != train1.lsvc_predict)]

In [ ]:
prf_own = metrics.precision_recall_fscore_support(train1.fraud, train1.own_predict, beta=0.5172, average='binary')
prf_xgb = metrics.precision_recall_fscore_support(train1.fraud, train1.xgb_predict, beta=0.5172, average='binary')
prf_lsvc = metrics.precision_recall_fscore_support(train1.fraud, train1.lsvc_predict, beta=0.5172, average='binary')
dmc_score_own = np.sum(metrics.confusion_matrix(train1.fraud, train1.own_predict)* np.array([[0, -25], [-5, 5]]))
dmc_score_xgb = np.sum(metrics.confusion_matrix(train1.fraud, train1.xgb_predict)* np.array([[0, -25], [-5, 5]]))
dmc_score_lsvc = np.sum(metrics.confusion_matrix(train1.fraud, train1.lsvc_predict)* np.array([[0, -25], [-5, 5]]))


print("OWN CLASSIFIER: Precision: {} Recall: {} Fbeta: {} DMC: {}".format(prf[0], prf[1], prf[2],dmc_score_own))
print("XGBOOST: Precision: {} Recall: {} Fbeta: {} DMC: {}".format(prf_xgb[0], prf_xgb[1], prf_xgb[2],dmc_score_xgb))
print("LINEAR SVCPrecision: {} Recall: {} Fbeta: {} DMC: {}".format(prf_lsvc[0], prf_lsvc[1], prf_lsvc[2],dmc_score_lsvc))

### 7.) pseudo-label the test set and create new classifier based on this
first we predict the original test data labels using the new extended pltrain from above cell and second we use this test data labels to train a new classifier

### 8.) evaluate our new classifier with the original training set

### 9.) combine the pseudo labeled test set with the original train data to train our final classifier

### 10.) predict labels for the test set using our final classifier

### 11.) generate output file neeeded for submission

### 12.) evaluate our new classifier with the validation set
Now at the very end we can also test our final model on a validation split never used before. just for comparison. 

**For the final submission, the following code should will not be run and the full train (incl. this val split) set will be used above**